In [16]:
docs = ['오늘 동물원에서 원숭이와 코끼리를 봤어',
       '동물원에서 원숭이에게 바나나를 줬어 바나나를']


In [17]:
doc_Is= [ ]
for doc in docs:
    doc_Is.append(doc.split())
doc_Is


[['오늘', '동물원에서', '원숭이와', '코끼리를', '봤어'],
 ['동물원에서', '원숭이에게', '바나나를', '줬어', '바나나를']]

In [ ]:
# TF 계산
# IDF 계산
# TF- IDF 계산

In [ ]:
# TF-IDF 계산절차

# 1.토큰 인덱스 생성 

In [18]:
from collections import defaultdict

#빈딕셔너리 
word2id = defaultdict(lambda : len(word2id))
word2id

defaultdict(<function __main__.<lambda>()>, {})

In [19]:
for doc in doc_Is:
    for token in doc:
        word2id[token]
word2id
# 빈딕셔너리에다가 토큰들을 넣어주는것. 

defaultdict(<function __main__.<lambda>()>,
            {'오늘': 0,
             '동물원에서': 1,
             '원숭이와': 2,
             '코끼리를': 3,
             '봤어': 4,
             '원숭이에게': 5,
             '바나나를': 6,
             '줬어': 7})

In [20]:
 #매트릭스 먼저 만들어주기 
# dtm, tdm 차이는 해당 라이브러리마다 쓰는 것이 다름 -> 차이는 거의 없다..? 

import numpy as np

# 기본 0인 행렬 
DTM = np.zeros((len(doc_Is),len(word2id)), dtype=int)
print(DTM)
for i, doc in enumerate(doc_Is):
    for token in doc:
        DTM[i, word2id[token]] += 1 # 해당 토큰의 위치  
        #자기에 인덱스에 맞는 것들에다가 +1 
        
DTM

[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]


array([[1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 2, 1]])

In [21]:
DTM[0].sum()

5

In [22]:
# DTM을 만들고 TF,IDF 만들기 
def computeTF(DTM):
    doc_len = len(DTM) # 문서의 갯수 2개 
    word_len = len(DTM[0]) #토큰의 개수 8개 
    
    #TF 계산하기 전에 0으로 셋팅 
    tf = np.zeros((doc_len, word_len))
    #print(tf)
    
    #TF 계산 특정단어등장빈도/문서내 전체등장단어빈도 
    
    for doc_i in range(doc_len) : #문서를 도는 FOR문
        for word_i in range(word_len) :#단어를 도는 FOR문 
            tf[doc_i, word_i] = DTM[doc_i, word_i]/DTM[doc_i].sum()
            
        return tf

In [23]:
computeTF(DTM) #계산 값 틀림 

array([[0.2, 0.2, 0.2, 0.2, 0.2, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ]])

In [24]:
print(DTM)

[[1 1 1 1 1 0 0 0]
 [0 1 0 0 0 1 2 1]]


In [28]:
import math

# IDF 계산: log(총문서수/단어가 등장한 문서수)

def computeIDF(DTM):
    doc_len = len(DTM)
    word_len = len(DTM[0])
    
    idf = np.zeros(word_len)
    
    for i in range(word_len) :
        idf[i] = math.log10(doc_len / np.count_nonzero(DTM[:,i]))  # 열이 0이아닌 것의 갯수 세기 
        
    return idf

In [29]:
computeIDF(DTM)

array([0.30103, 0.     , 0.30103, 0.30103, 0.30103, 0.30103, 0.30103,
       0.30103])

In [30]:
# TF-IDF 곱

def computeTFIDF(DTM):
    tf = computeTF(DTM)
    idf = computeIDF(DTM)
    tfidf = np.zeros(tf.shape)
    for doc_i in range(tf.shape[0]) : #문서
        for word_i in range(tf.shape[1]) : #단어 
            tfidf[doc_i, word_i] = tf[doc_i, word_i] * idf[word_i]
            
    return tfidf
    

In [31]:
computeTFIDF(DTM)

array([[0.060206, 0.      , 0.060206, 0.060206, 0.060206, 0.      ,
        0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      ]])

In [32]:
import pandas as pd

sorted_vocab = sorted((value, key) for key, value in word2id.items())
print(sorted_vocab)

vocab = [v[1] for v in sorted_vocab]
print(vocab)

[(0, '오늘'), (1, '동물원에서'), (2, '원숭이와'), (3, '코끼리를'), (4, '봤어'), (5, '원숭이에게'), (6, '바나나를'), (7, '줬어')]
['오늘', '동물원에서', '원숭이와', '코끼리를', '봤어', '원숭이에게', '바나나를', '줬어']


In [34]:
tfidf = computeTFIDF(DTM)
pd.DataFrame(tfidf, columns=vocab)

,오늘,동물원에서,원숭이와,코끼리를,봤어,원숭이에게,바나나를,줬어
0,0.060206,0.0,0.060206,0.060206,0.060206,0.0,0.0,0.0
1,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0


## sklearn 

In [9]:
docs = docs = ['오늘 동물원에서 원숭이와 코끼리를 봤어',
       '동물원에서 원숭이에게 바나나를 줬어 바나나를']

from sklearn.feature_extraction.text import TfidfVectorizer
#선언 
tfidv = TfidfVectorizer()
tfidv = tfidv.fit(docs)
tfidv.transform(docs).toarray()



array([[0.33517574, 0.        , 0.47107781, 0.47107781, 0.        ,
        0.47107781, 0.        , 0.47107781],
       [0.27894255, 0.78408803, 0.        , 0.        , 0.39204401,
        0.        , 0.39204401, 0.        ]])

In [12]:
vocab = tfidv.get_feature_names()
vocab

['동물원에서', '바나나를', '봤어', '오늘', '원숭이에게', '원숭이와', '줬어', '코끼리를']

In [7]:
vocab

['동물원에서', '바나나를', '봤어', '오늘', '원숭이에게', '원숭이와', '줬어', '코끼리를']

In [8]:
tfidv.transform(docs).toarray()[0]

array([0.33517574, 0.        , 0.47107781, 0.47107781, 0.        ,
       0.47107781, 0.        , 0.47107781])

In [36]:
import pandas as pd
df = pd.DataFrame(tfidv.transform(docs).toarray(),columns = vocab)
df

,오늘,동물원에서,원숭이와,코끼리를,봤어,원숭이에게,바나나를,줬어
0,0.335176,0.000000,0.471078,0.471078,0.000000,0.471078,0.000000,0.471078
1,0.278943,0.784088,0.000000,0.000000,0.392044,0.000000,0.392044,0.000000


## gensim 

In [46]:
import gensim
from gensim import corpora
from gensim.models import TfidfModel

docs = ['오늘 동물원에서 원숭이와 코끼리를 봤어', 
        '동물원에서 원숭이에게 바나나를 줬어 바나나를']

doc_Is = [doc.split() for doc in docs] #공백으로 토큰화
id2word = corpora.Dictionary(doc_Is)
TDM = [id2word.doc2bow(doc) for doc in doc_Is]
model = TfidfModel(TDM)
tfidf = model[TDM]
tfidf[0]

[(1, 0.5), (2, 0.5), (3, 0.5), (4, 0.5)]

In [47]:
import pandas as pd 
from gensim.matutils import sparse2full

vocab = [id2word[i] for i in id2word.keys()]
TDM_matrix = [sparse2full(doc, len(vocab)).tolist() for doc in tfidf]
pd.DataFrame(TDM_matrix, columns=vocab)

,동물원에서,봤어,오늘,원숭이와,코끼리를,바나나를,원숭이에게,줬어
0,0.0,0.5,0.5,0.5,0.5,0.000000,0.000000,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.816497,0.408248,0.408248
